# عوامل AutoGen در محیط تولید: مشاهده‌پذیری و ارزیابی

در این آموزش، یاد می‌گیریم که چگونه **گام‌های داخلی (ردیابی‌ها) عوامل [Autogen](https://github.com/microsoft/autogen)** را **نظارت کنیم** و با استفاده از [Langfuse](https://langfuse.com) **عملکرد آن را ارزیابی کنیم**.

این راهنما معیارهای ارزیابی **آنلاین** و **آفلاین** را که توسط تیم‌ها برای آوردن عوامل به محیط تولید به‌صورت سریع و قابل‌اعتماد استفاده می‌شود، پوشش می‌دهد.

**چرا ارزیابی عوامل هوش مصنوعی مهم است:**
- رفع اشکال در مواقعی که وظایف شکست می‌خورند یا نتایج نامطلوب تولید می‌کنند
- نظارت بر هزینه‌ها و عملکرد در زمان واقعی
- بهبود قابلیت اطمینان و ایمنی از طریق بازخورد مداوم


## مرحله ۱: تنظیم متغیرهای محیطی

کلیدهای API مربوط به Langfuse را با ثبت‌نام در [Langfuse Cloud](https://cloud.langfuse.com/) یا [میزبانی شخصی Langfuse](https://langfuse.com/self-hosting) دریافت کنید.

_**توجه:** افرادی که از میزبانی شخصی استفاده می‌کنند، می‌توانند از [ماژول‌های Terraform](https://langfuse.com/self-hosting/azure) برای استقرار Langfuse در Azure استفاده کنند. به‌طور جایگزین، می‌توانید Langfuse را با استفاده از [نمودار Helm](https://langfuse.com/self-hosting/kubernetes-helm) روی Kubernetes مستقر کنید._


In [5]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

با تنظیم متغیرهای محیطی، اکنون می‌توانیم مشتری Langfuse را مقداردهی اولیه کنیم. `get_client()` مشتری Langfuse را با استفاده از اطلاعات ورود ارائه‌شده در متغیرهای محیطی مقداردهی اولیه می‌کند.


In [6]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## مرحله ۲: مقداردهی اولیه ابزار OpenLit

حالا ابزار [OpenLit](https://github.com/openlit/openlit) را مقداردهی اولیه می‌کنیم. OpenLit به‌صورت خودکار عملیات AutoGen را ضبط کرده و بازه‌های OpenTelemetry (OTel) را به Langfuse صادر می‌کند.


In [7]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True, disabled_instrumentors=["mistral"])

## مرحله ۳: اجرای عامل خود

اکنون یک عامل چند مرحله‌ای تنظیم می‌کنیم تا ابزارسازی خود را آزمایش کنیم.


In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": False
    },
)

In [8]:
# 🍴 Agent 1 – proposes ONE healthy meal idea each turn
meal_planner_agent = AssistantAgent(
    "meal_planner_agent",
    model_client=client,
    description="A seasoned meal-planning coach who suggests balanced meals.",
    system_message="""
    You are a Meal-Planning Assistant with a decade of experience helping busy people prepare meals.
    Goal: propose the single best meal (breakfast, lunch, or dinner) given the user's context.
    Each response must contain ONLY one complete meal idea (title + very brief component list) — no extras.
    Keep it concise: skip greetings, chit-chat, and filler.
    """,
)

# 🥗 Agent 2 – checks nutritional quality & variety
nutritionist_agent = AssistantAgent(
    "nutritionist_agent",
    model_client=client,
    description="A registered dietitian ensuring meals meet nutritional standards.",
    system_message="""
    You are a Nutritionist focused on whole-food, macro-balanced eating.
    Evaluate the meal_planner_agent’s recommendation.
    If the meal is nutritionally sound, sufficiently varied, and portion-appropriate, respond with 'APPROVE'.
    Otherwise, give high-level guidance on how to improve it (e.g. 'add a plant-based protein') — do NOT provide a full alternative recipe.
    """,
)

In [9]:
# ✅ Chat stops once the nutritionist says APPROVE
termination = TextMentionTermination("APPROVE")

# 🔄 Alternate turns between the two agents until termination
team = RoundRobinGroupChat(
    [meal_planner_agent, nutritionist_agent],
    termination_condition=termination,
)

# Example kickoff
user_input = "I'm looking for a quick, delicious dinner I can prep after work. I have 30 minutes and minimal clean-up is ideal."

In [ ]:
with langfuse.start_as_current_span(name="create_meal_plan") as span:
    async for message in team.run_stream(task=user_input):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

# Flush the trace to Langfuse for short-lived environments such as Jupyter Notebooks
langfuse.flush()

### ساختار ردیابی

Langfuse یک **ردیابی** را ثبت می‌کند که شامل **اسپن‌ها** است، که هر کدام نمایانگر یک مرحله از منطق عامل شما هستند. در اینجا، ردیابی شامل اجرای کلی عامل و زیر-اسپن‌هایی برای:
- عامل برنامه‌ریز وعده غذایی
- عوامل متخصص تغذیه

شما می‌توانید این موارد را بررسی کنید تا دقیقاً ببینید زمان کجا صرف شده، چند توکن استفاده شده و موارد دیگر:

![درخت ردیابی در Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/trace-tree.png)

_[لینک به ردیابی](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


## ارزیابی آنلاین

ارزیابی آنلاین به معنای ارزیابی عامل در یک محیط زنده و واقعی است، یعنی در زمان استفاده واقعی در تولید. این فرآیند شامل نظارت بر عملکرد عامل در تعاملات واقعی کاربران و تحلیل مداوم نتایج می‌شود.

### معیارهای رایج برای پیگیری در تولید

1. **هزینه‌ها** — ابزارهای اندازه‌گیری میزان استفاده از توکن‌ها را ثبت می‌کنند، که می‌توانید با اختصاص دادن قیمت به هر توکن، آن را به هزینه‌های تقریبی تبدیل کنید.
2. **زمان تأخیر** — زمان لازم برای تکمیل هر مرحله یا کل فرآیند را مشاهده کنید.
3. **بازخورد کاربران** — کاربران می‌توانند بازخورد مستقیم (مثبت/منفی) ارائه دهند تا به بهبود یا اصلاح عامل کمک کنند.
4. **LLM به‌عنوان قاضی** — از یک LLM جداگانه برای ارزیابی خروجی عامل خود در زمان نزدیک به واقعی استفاده کنید (مثلاً بررسی برای عدم توهین یا صحت).

در ادامه، نمونه‌هایی از این معیارها نشان داده شده است.


#### ۱. هزینه‌ها

در تصویر زیر استفاده از فراخوانی‌های `gpt-4o-mini` نمایش داده شده است. این تصویر برای مشاهده مراحل پرهزینه و بهینه‌سازی عامل شما مفید است.

![هزینه‌ها](https://langfuse.com/images/cookbook/example-autogen-evaluation/gpt-4o-costs.png)

_[لینک به ردیابی](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


#### 2. تأخیر

ما همچنین می‌توانیم ببینیم هر مرحله چقدر زمان برده است. در مثال زیر، کل اجرا حدود ۳ ثانیه طول کشیده که می‌توانید آن را به مراحل مختلف تقسیم کنید. این به شما کمک می‌کند نقاط ضعف را شناسایی کرده و عامل خود را بهینه کنید.

![تأخیر](https://langfuse.com/images/cookbook/example-autogen-evaluation/agent-latency.png) 

_[لینک به ردگیری](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64?display=timeline)_


#### 3. بازخورد کاربران

اگر عامل شما در یک رابط کاربری تعبیه شده باشد، می‌توانید بازخورد مستقیم کاربران (مانند علامت مثبت/منفی در یک رابط چت) را ثبت کنید.


In [10]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="autogen-request-user-feedback-1") as span:
    
    async for message in team.run_stream(task="Create a meal with potatoes"):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was delicious, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:
    # ... Autogen execution ...

    async for message in team.run_stream(task="I am allergic to gluten."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )

id='da068880-22ae-4f01-9f01-2bb231939089' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 43, 732669, tzinfo=datetime.timezone.utc) content='Create a meal with potatoes' type='TextMessage'
id='ad937ce4-3534-493f-824b-ca9c226b5287' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=95, completion_tokens=30) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 186423, tzinfo=datetime.timezone.utc) content='Potato and Spinach Frittata  \n- Eggs  \n- Potatoes  \n- Fresh spinach  \n- Onion  \n- Cheese (optional)  ' type='TextMessage'
id='50fd33c1-057f-49fe-afad-ee86d164296d' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=132, completion_tokens=4) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 581059, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
id='e371de6c-e5fc-42c1-8eda-e5b8cd5accab' source='user' models_usage=None met

In [ ]:
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined_trace_id",
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

بازخورد کاربران سپس در Langfuse ثبت می‌شود:

![بازخورد کاربران در حال ثبت شدن در Langfuse است](https://langfuse.com/images/cookbook/example-autogen-evaluation/user-feedback.png)


#### ۴. امتیازدهی خودکار با استفاده از LLM به‌عنوان قاضی

LLM به‌عنوان قاضی یکی دیگر از روش‌های ارزیابی خودکار خروجی عامل شما است. شما می‌توانید یک فراخوان جداگانه به LLM تنظیم کنید تا درستی، سمی بودن، سبک یا هر معیار دیگری که برایتان مهم است را ارزیابی کند.

**روند کار**:
1. شما یک **الگوی ارزیابی** تعریف می‌کنید، مثلاً: "بررسی کنید که آیا متن سمی است."
2. مدلی را که به‌عنوان مدل قاضی استفاده می‌شود تنظیم می‌کنید؛ در این مثال `gpt-4o-mini` که از طریق Azure پرس‌وجو می‌شود.
3. هر بار که عامل شما خروجی تولید می‌کند، آن خروجی را با الگو به LLM قاضی ارسال می‌کنید.
4. LLM قاضی با یک امتیاز یا برچسب پاسخ می‌دهد که آن را در ابزار مشاهده‌پذیری خود ثبت می‌کنید.

مثالی از Langfuse:

![ارزیاب LLM به‌عنوان قاضی](https://langfuse.com/images/cookbook/example-autogen-evaluation/evaluator.png)


In [12]:
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:

    async for message in team.run_stream(task="I am a picky eater and not sure if you find something for me."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message) 

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

langfuse.flush()

id='eefc628d-502f-451a-8f70-be486f62f8c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 29, 171393, tzinfo=datetime.timezone.utc) content='I am a picky eater and not sure if you find something for me.' type='TextMessage'
id='13b3e14b-bcf7-42a5-80d6-54b0c7be765e' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=352, completion_tokens=27) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 30, 433516, tzinfo=datetime.timezone.utc) content='Chicken Alfredo Pasta  \n- Gluten-free pasta  \n- Grilled chicken breast  \n- Heavy cream  \n- Parmesan cheese  \n- Garlic  ' type='TextMessage'
id='550f2dee-0e08-4bbd-b67f-991b467328f1' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=386, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 31, 505173, tzinfo=datetime.timezone.utc) content='Consider incorporating some vegetables, like spinach or broccoli, to increase the nutrien

شما می‌توانید ببینید که پاسخ این مثال به عنوان "غیرسمی" ارزیابی شده است.

![امتیاز ارزیابی LLM به عنوان قاضی](https://langfuse.com/images/cookbook/example-autogen-evaluation/llm-as-a-judge-score.png)


#### ۵. نمای کلی معیارهای مشاهده‌پذیری

تمام این معیارها را می‌توان به‌صورت یکجا در داشبوردها مشاهده کرد. این امکان را فراهم می‌کند که به‌سرعت عملکرد عامل خود را در جلسات مختلف بررسی کنید و به مرور زمان معیارهای کیفیت را دنبال کنید.

![نمای کلی معیارهای مشاهده‌پذیری](https://langfuse.com/images/cookbook/example-autogen-evaluation/dashboard.png)


## ارزیابی آفلاین

ارزیابی آنلاین برای دریافت بازخورد زنده ضروری است، اما شما همچنین به **ارزیابی آفلاین** نیاز دارید—بررسی‌های سیستماتیک قبل یا در طول توسعه. این کار به حفظ کیفیت و اطمینان کمک می‌کند قبل از اعمال تغییرات در محیط تولید.


### ارزیابی مجموعه داده

در ارزیابی آفلاین، معمولاً:
1. یک مجموعه داده معیار دارید (شامل جفت‌های پرسش و پاسخ مورد انتظار)
2. عامل خود را روی آن مجموعه داده اجرا می‌کنید
3. خروجی‌ها را با نتایج مورد انتظار مقایسه می‌کنید یا از یک مکانیزم امتیازدهی اضافی استفاده می‌کنید

در ادامه، این روش را با استفاده از [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset) که شامل پرسش‌ها و پاسخ‌های مورد انتظار است، نشان می‌دهیم.


In [16]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

/Users/jannik/Documents/GitHub/ai-agents-for-beginners/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to t

سپس، یک موجودیت مجموعه داده در Langfuse ایجاد می‌کنیم تا اجراها را دنبال کنیم. سپس، هر مورد از مجموعه داده را به سیستم اضافه می‌کنیم.


In [17]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_autogen-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

Dataset(id='cmcm7524d00kjad07s2cjwqcf', name='qa-dataset_autogen-agent', description='q&a dataset uploaded from Hugging Face', metadata={'date': '2025-03-21', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc))

In [18]:
df_25 = df.sample(25) # For this example, we upload only 25 dataset questions

for idx, row in df_25.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![موارد مجموعه داده در Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/example-dataset.png)


### اجرای عامل بر روی مجموعه داده

ابتدا، یک عامل ساده Autogen را ایجاد می‌کنیم که با استفاده از مدل‌های Azure OpenAI به سؤالات پاسخ می‌دهد.


In [8]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    max_tokens=5000,
    model_info={
        "json_output": True,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
        "structured_output": True,
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

In [18]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are participant in a quizz show and you are given a question. You need to create a short answer to the question.",
)

سپس، یک تابع کمکی `my_agent()` تعریف می‌کنیم.


In [19]:
async def my_agent(user_query: str):

    with langfuse.start_as_current_span(name="autogen-trace") as span:

        # Execute the agent response
        response = await agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken(),
        )

        span.update_trace(
            input=user_query,
            output=response.chat_message.content,
        )

    return str(response.chat_message.content)

# Test the function
await my_agent("What is the capital of France?")

'The capital of France is Paris.'

در نهایت، ما روی هر آیتم مجموعه داده حلقه می‌زنیم، عامل را اجرا می‌کنیم و ردگیری را به آیتم مجموعه داده متصل می‌کنیم. در صورت تمایل می‌توانیم یک امتیاز ارزیابی سریع نیز اضافه کنیم.


In [20]:
dataset_name = "qa-dataset_autogen-agent"
current_run_name = "dev_tasks_run-autogen_gpt-4.1" # Identifies this specific evaluation run
current_run_metadata={"model_provider": "Azure", "model": "gpt-4.1"}
current_run_description="Evaluation run for Autogen model on July 3rd"

dataset = langfuse.get_dataset('qa-dataset_autogen-agent')

for item in dataset.items:
    print(f"Running evaluation for item: {item.id} (Input: {item.input})")
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata=current_run_metadata,
        run_description=current_run_description
    ) as root_span: 
        # All subsequent langfuse operations within this block are part of this trace.
        generated_answer = await my_agent(user_query = item.input["text"])
    
    print("Generated Answer: ", generated_answer)
 
print(f"\nFinished processing dataset '{dataset_name}' for run '{current_run_name}'.")

langfuse.flush()

Running evaluation for item: 09810cc4-9992-4712-a3b2-7224da31776a (Input: {'text': 'In Hindu mythology, which deity is the Ganges river dolphin associated with?'})
Generated Answer:  In Hindu mythology, the Ganges river dolphin is associated with the deity Ganga.
Running evaluation for item: bb113f94-7723-47c6-8c34-59d883044514 (Input: {'text': 'What significant discovery did the LHCb collaboration report in 2015?'})
Generated Answer:  In 2015, the LHCb collaboration reported the discovery of pentaquark particles.
Running evaluation for item: 4d8ae54e-ceab-46d0-ad2c-6e8e223589a9 (Input: {'text': 'What is the MÄ\x81ori name for the red-crowned parakeet?'})
Generated Answer:  The Māori name for the red-crowned parakeet is kākāriki.
Running evaluation for item: 21e5a0d5-f619-4a73-868e-9955053b3e72 (Input: {'text': 'Who starred in the 1978 television film adaptation of Les MisÃ©rables?'})
Generated Answer:  Richard Jordan starred as Jean Valjean in the 1978 television film adaptation of Le

می‌توانید این فرآیند را با پیکربندی‌های مختلف عامل تکرار کنید، مانند:

- مدل‌ها (gpt-4o-mini، gpt-4.1 و غیره)
- درخواست‌ها (prompts)
- ابزارها (جستجو در مقابل بدون جستجو)
- پیچیدگی عامل (چندعاملی در مقابل تک‌عاملی)

سپس آن‌ها را در Langfuse کنار هم مقایسه کنید. در این مثال، من عامل را ۳ بار روی ۲۵ سؤال مجموعه داده اجرا کردم. برای هر اجرا، از یک مدل Azure OpenAI متفاوت استفاده کردم. می‌توانید ببینید که تعداد سؤالات پاسخ داده شده به‌درستی هنگام استفاده از یک مدل بزرگ‌تر بهبود می‌یابد (همان‌طور که انتظار می‌رود). امتیاز `correct_answer` توسط یک [ارزیاب LLM-as-a-Judge](https://langfuse.com/docs/scores/model-based-evals) ایجاد شده است که برای قضاوت درستی سؤال بر اساس پاسخ نمونه داده شده در مجموعه داده تنظیم شده است.

![نمای کلی اجرای مجموعه داده](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset_runs.png)
![مقایسه اجرای مجموعه داده](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset-run-comparison.png)



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادقتی‌هایی باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما هیچ مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
